In [1]:
!pip install keras --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report  # For evaluation

In [3]:
import pandas as pd
import numpy as np
import os
from collections import Counter

def load_and_preprocess_data(directory):
    data = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)

            num_time_steps = 480
            num_features = df.shape[1] -1 # Number of features (excluding the label)
            num_segments = len(df) // num_time_steps  # Calculate the number of full 480-row segments

            for i in range(num_segments):
                start_idx = i * num_time_steps
                end_idx = (i + 1) * num_time_steps
                segment_df = df.iloc[start_idx:end_idx] # Extract a 480-row segment

                # --- Label Selection Logic (within each segment) ---
                label_counts = Counter(segment_df['Sleep_Stage'])
                lowest_count_label = label_counts.most_common()[-1][0]
                segment_df_filtered = segment_df[segment_df['Sleep_Stage'] == lowest_count_label]

                if not segment_df_filtered.empty: #Check if the label existed in the segment
                    label = segment_df_filtered['Sleep_Stage'].iloc[0]
                    features = segment_df_filtered.drop('Sleep_Stage', axis=1)
                    reshaped_features = np.array(features).reshape(1, num_time_steps, num_features) # Reshape for single segment

                    data.append([reshaped_features[0], label]) # Append the segment and its label

    df_final = pd.DataFrame(data, columns=['features', 'label'])
    return df_final

In [4]:
def load_and_preprocess_data_test(directory):
    directory = "/kaggle/input/io-t-sleep-stage-classification-version-2/test_segment/test_segment"
    data = []
    for folder in os.listdir(directory):
        path = os.path.join(directory,folder)
        for filename in os.listdir(path):
            if filename.endswith(".csv"):
                filepath = os.path.join(path, filename)
                name = filepath.split('.csv')[0]
                df = pd.read_csv(filepath)
                label = '' # All rows in a 30 sec segment have the same label
                features = df
                # Reshape to (samples, time_steps, features)
                # 16 Hz * 30 seconds = 480 time steps per segment
                num_time_steps = 480
                num_features = features.shape[1]
                reshaped_features = np.array(features).reshape(-1, num_time_steps, num_features) # -1 infers number of samples
                data.append([name,reshaped_features[0]])
            

    df_final = pd.DataFrame(data, columns=['id','features'])
    return df_final

In [5]:

train_dir = "/kaggle/input/io-t-sleep-stage-classification-version-2/train/train"  # Path to your training data folder
test_segment_dir = "/kaggle/input/io-t-sleep-stage-classification-version-2/test_segment/test_segment/test001" # Path to your test segments data
submission_file = "sample_submission.csv"

train_data = load_and_preprocess_data(train_dir)



In [6]:
train_data.label.unique()

array(['W', 'N', 'R'], dtype=object)

In [7]:
train_data[train_data.label == 'R']

,features,label
144,"[[-0.6929213388638342, -28.563517861705893, -3...",R
145,"[[-70.41982434858774, -28.65922219127341, -36....",R
146,"[[30.840163680674085, -28.71542383497956, -36....",R
147,"[[32.253061766875646, -28.672256841291503, -36...",R
148,"[[-3.0849055195987773, -28.324518154686427, -3...",R
...,...,...
65747,"[[11.971762269761369, -60.30173877407333, -15....",R
65748,"[[24.58346023731236, -60.30173877407333, -15.8...",R
65749,"[[20.01616908200901, -60.30173877407333, -15.8...",R
65750,"[[-22.09964431649612, -60.30173877797718, -15....",R


In [8]:
from sklearn.preprocessing import StandardScaler, LabelEncoder 
X = np.array(list(train_data['features']))
y = np.array(train_data['label'])
le = LabelEncoder()
y = le.fit_transform(y) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) # 30% for temp (validation + test)

In [9]:
train_data

,features,label
0,"[[-16.787331661121932, -29.65461956713005, -35...",W
1,"[[-87.70960221690167, -30.33420625694373, -34....",W
2,"[[-61.13355054611345, -30.644801909392143, -34...",W
3,"[[-38.19160502993879, -29.97652578795709, -34....",W
4,"[[38.93285066688821, -30.78044170358605, -34.5...",W
...,...,...
66468,"[[122.12527938638864, -16.480066280328437, -36...",W
66469,"[[-69.19549169088637, -18.686316932516377, -36...",W
66470,"[[-17.473833641543482, -13.796254236720072, -3...",W
66471,"[[112.57196061426691, -28.73897913614809, -29....",W


In [10]:

scaler = StandardScaler()

# Reshape X_train for scaling (samples * timesteps * features) -> (samples * features* timesteps)
X_train_reshaped = X_train.reshape(-1, X_train.shape[2] * X_train.shape[1])
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_train_scaled = X_train_scaled.reshape(-1, X_train.shape[1], X_train.shape[2]) # back to original shape

X_test_reshaped = X_test.reshape(-1, X_test.shape[2] * X_test.shape[1])
X_test_scaled = scaler.transform(X_test_reshaped)  # Use the same scaler fitted on training data
X_test_scaled = X_test_scaled.reshape(-1, X_test.shape[1], X_test.shape[2])

num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)




In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

def attention_block(inputs):
    """Self-Attention for LSTM"""
    attention = layers.MultiHeadAttention(num_heads=4, key_dim=64)(inputs, inputs)
    attention = layers.Add()([inputs, attention])  # Residual Connection
    return layers.LayerNormalization()(attention)

input_layer = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))

# BiLSTM Layers
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.001)))(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Bidirectional(layers.LSTM(256, return_sequences=True, kernel_regularizer=regularizers.l2(0.001)))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

# Attention Layer
x = attention_block(x)

# Final LSTM Layer before output
x = layers.Bidirectional(layers.LSTM(128, return_sequences=False, kernel_regularizer=regularizers.l2(0.001)))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

# Fully Connected Layer
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)

output_layer = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=input_layer, outputs=output_layer)

# Optimizer: AdamW + Cosine Decay
initial_lr = 0.001
lr_schedule = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=initial_lr, decay_steps=10000
)
optimizer = keras.optimizers.AdamW(learning_rate=lr_schedule, weight_decay=1e-4)

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 480, 8)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 480, 256)       │        140,288 │ input_layer[0][0]      │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 480, 256)       │          1,024 │ bidirectional[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 480, 256)       │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 480, 512)       │      1,050,624 │ dropout[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 480, 512)       │          2,048 │ bidirectional_1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 480, 512)       │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 480, 512)       │        525,568 │ dropout_1[0][0],       │
│ (MultiHeadAttention)      │                        │                │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 480, 512)       │              0 │ dropout_1[0][0],       │
│                           │                        │                │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 480, 512)       │          1,024 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 256)            │        656,384 │ layer_normalization[0… │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 256)            │          1,024 │ bidirectional_2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 256)            │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │         65,792 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 256)            │              0 │ dense[0][0]            │
├──────────────────────

 Total params: 2,444,547 (9.33 MB)

 Trainable params: 2,442,499 (9.32 MB)

 Non-trainable params: 2,048 (8.00 KB)

In [12]:
# 4. Train the Model
epochs = 100
batch_size = 100

history = model.fit(X_train_scaled, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)



Epoch 1/100
539/539 ━━━━━━━━━━━━━━━━━━━━ 266s 471ms/step - accuracy: 0.5874 - loss: 2.5744 - val_accuracy: 0.6652 - val_loss: 1.5144
Epoch 2/100
539/539 ━━━━━━━━━━━━━━━━━━━━ 260s 482ms/step - accuracy: 0.6519 - loss: 1.4000 - val_accuracy: 0.6239 - val_loss: 1.1826
Epoch 3/100
539/539 ━━━━━━━━━━━━━━━━━━━━ 260s 482ms/step - accuracy: 0.6697 - loss: 1.0774 - val_accuracy: 0.6816 - val_loss: 0.9867
Epoch 4/100
539/539 ━━━━━━━━━━━━━━━━━━━━ 259s 480ms/step - accuracy: 0.6674 - loss: 1.0133 - val_accuracy: 0.6656 - val_loss: 0.9893
Epoch 5/100
539/539 ━━━━━━━━━━━━━━━━━━━━ 259s 480ms/step - accuracy: 0.6757 - loss: 0.9519 - val_accuracy: 0.6869 - val_loss: 0.8862
Epoch 6/100
539/539 ━━━━━━━━━━━━━━━━━━━━ 259s 481ms/step - accuracy: 0.6739 - loss: 0.9082 - val_accuracy: 0.6799 - val_loss: 0.8905
Epoch 7/100
539/539 ━━━━━━━━━━━━━━━━━━━━ 259s 481ms/step - accuracy: 0.6673 - loss: 0.9316 - val_accuracy: 0.5477 - val_loss: 1.1510
Epoch 8/100
539/539 ━━━━━━━━━━━━━━━━━━━━ 260s 483ms/step - accuracy: 

In [13]:
# 5. Evaluate the Model (More detailed evaluation)
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get predicted class labels
y_true_classes = np.argmax(y_test, axis=1) # Get true class labels

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

class_report = classification_report(y_true_classes, y_pred_classes)
print("Classification Report:\n", class_report)



Test Loss: 0.8705135583877563
Test Accuracy: 0.7107400894165039
208/208 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step
Confusion Matrix:
 [[4212    0  214]
 [ 650    0   15]
 [1044    0  513]]
Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.95      0.82      4426
           1       0.00      0.00      0.00       665
           2       0.69      0.33      0.45      1557

    accuracy                           0.71      6648
   macro avg       0.47      0.43      0.42      6648
weighted avg       0.64      0.71      0.65      6648



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
test_data = load_and_preprocess_data_test(test_segment_dir)

test_data

,id,features
0,/kaggle/input/io-t-sleep-stage-classification-...,"[[-8.26647136847187, 59.31294859734474, -10.87..."
1,/kaggle/input/io-t-sleep-stage-classification-...,"[[-33.737309672712634, -32.004096055393184, 38..."
2,/kaggle/input/io-t-sleep-stage-classification-...,"[[0.9678524725302452, -47.19056679892199, -22...."
3,/kaggle/input/io-t-sleep-stage-classification-...,"[[0.108033853352705, -24.74665457216458, 50.43..."
4,/kaggle/input/io-t-sleep-stage-classification-...,"[[40.0336184993943, 58.32472216520449, -16.846..."
...,...,...
7005,/kaggle/input/io-t-sleep-stage-classification-...,"[[10.562214461978355, -42.16635811486883, 2.96..."
7006,/kaggle/input/io-t-sleep-stage-classification-...,"[[-8.910720370054893, -40.18434756956151, 2.96..."
7007,/kaggle/input/io-t-sleep-stage-classification-...,"[[72.01702354399089, -46.51449536994952, 39.53..."
7008,/kaggle/input/io-t-sleep-stage-classification-...,"[[-94.06339813082532, -56.34752639546982, 14.8..."


In [15]:
id_list = test_data.id
test_list = np.array(list(test_data['features']))

X_pred_reshaped = test_list.reshape(-1, test_list.shape[2] * test_list.shape[1])
X_pred_scaled = scaler.fit_transform(X_pred_reshaped)
X_pred_scaled = X_pred_scaled.reshape(-1, test_list.shape[1], test_list.shape[2]) # back to original shape



In [16]:
id_list = [x.split('/')[-1] for x in id_list]

In [17]:
test_predictions = model.predict(X_pred_scaled)

test_predictions

220/220 ━━━━━━━━━━━━━━━━━━━━ 16s 71ms/step


array([[0.4964074 , 0.11904844, 0.38454416],
       [0.73154116, 0.15144753, 0.11701135],
       [0.37417787, 0.10090926, 0.5249129 ],
       ...,
       [0.2793992 , 0.04588712, 0.6747137 ],
       [0.76630855, 0.06252386, 0.17116761],
       [0.5366009 , 0.23388173, 0.22951733]], dtype=float32)

In [18]:
predicted_labels = np.argmax(test_predictions, axis=1)
predicted_labels

array([0, 0, 2, ..., 2, 0, 0])

In [19]:
pred = le.inverse_transform(predicted_labels)
pred

array(['N', 'N', 'W', ..., 'W', 'N', 'N'], dtype=object)

In [20]:
temp_sub = pd.DataFrame({'id': id_list, 'labels': pred})

In [21]:
samp = pd.read_csv("/kaggle/input/io-t-sleep-stage-classification-version-2/sample_submission.csv")

In [22]:
samp

,id,labels
0,test001_00000,W
1,test001_00001,W
2,test001_00002,W
3,test001_00003,NaN
4,test001_00004,NaN
...,...,...
7005,test010_00696,NaN
7006,test010_00697,NaN
7007,test010_00698,NaN
7008,test010_00699,NaN


In [23]:
merged_df = pd.merge(samp, temp_sub, on='id', how='left')  # Example: Inner join on 'id'

In [24]:
merged_df.drop(columns = 'labels_x',inplace =True)
merged_df.rename(columns = {"labels_y" : "labels"} ,inplace =True)

In [25]:
merged_df.to_csv("sample_sub.csv",index=False)